#### DATA

In [1]:
import numpy as np
print(np.__version__)

1.26.4


In [1]:
import os
import json
import pandas as pd # Gzip 압축된 CSV 파일 불러오기

df = pd.read_csv("data.csv.gz", compression='gzip')
df.head()

,standard_form,dialect_form
0,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도
1,어어 그니까,어어 그니까
2,아 잘도,아 잘도
3,예초기 뱀도 짤려,예초기 뱀도 짤려
4,어 그니까 완전 끔찍해,어 그니까 완전 끔찍해


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2774257 entries, 0 to 2774256
Data columns (total 2 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   standard_form  object
 1   dialect_form   object
dtypes: object(2)
memory usage: 42.3+ MB


In [3]:
# 중복 제거
df_filtered = df[df["standard_form"] != df["dialect_form"]]
df_filtered = df_filtered.drop_duplicates()
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1116389 entries, 6 to 2774254
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   standard_form  1116389 non-null  object
 1   dialect_form   1116389 non-null  object
dtypes: object(2)
memory usage: 25.6+ MB


In [4]:
df_filtered.reset_index(drop=True, inplace=True)
df_filtered.head()

,standard_form,dialect_form
0,예전에 그렇게,예전에 겅
1,아 그러니까 예초 할땐 조심해야 돼,아 겅하니까이 예초 할땐 조심해야 돼
2,그러고 예초 벌초(하다그네)/(하다가),겅허고 예초 벌초(하다그네)/(하다가)
3,어 그거 튀겨서,어 그거 튀겨그네
4,돌멩이 해 가지고 실명된사람 있어,돌멩이 해 가지고 실명된사람 이서


In [1]:
import os
import json
import pandas as pd # Gzip 압축된 CSV 파일 불러오기

df_filtered = pd.read_csv("high_similarity_data.csv")
df_filtered.head()

,standard_form,dialect_form,similarity
0,막 그냥 예 습기가,막 그냥 예 습기가,1.0
1,그니까 제주도 사람 그런 이,그니까 제주도 사람 그런 이,1.0
2,그런데 이 지금은 비 오는 날 별로 안 좋아,그런데 이 지금은 비 오는 날 별로 안 좋아,1.0
3,그리고 안 되면은 이 그만둘 생각도 해야 돼,그리고 안 되면은 이 그만둘 생각도 해야 돼,1.0
4,왜냐면 이 난 겨울에 롱패딩 생활밖에 안 해,왜냐면 이 난 겨울에 롱패딩 생활밖에 안 해,1.0


In [4]:
df_low = pd.read_csv("low_similarity_data.csv")
df_low.head()
# len(df_low)

,standard_form,dialect_form,similarity
0,전봇대 잡아서 막 우는 게 비디오에 거기 나온 거야,전봇대 잡앙 막 우는 게 비디오에 거기 나온 거라,0.900000
1,꽃집에 갔다 왔어요 뭐하러 어두운 시간에 갔다왔습니까,꽃집에 갔다 왔수과 뭐하랜 어두운 시간에 갔다완마씨,0.900000
2,진짜 훠넌 거 진짜 통으로 사다가 먹으니까 따준다고했지,진짜 훠넌 거 진짜 통으로 사단 먹으난 따줄켄햄찌,0.900000
3,아 그렇구나 그러면 내가 말할게 내가 아는 건 뭐냐 하면,아 기구나 겨믄 내가 골아볼게 내가 아는 건 뭔고 하면,0.899999
4,그게 복이 되는줄을 아직도 몰라요,그게 복이 되는줄을 아직도 모름수과,0.899999


In [7]:
import re
SPECIALS = "".join([".", ",", ";", ":", "!", "?", '"', "'", "&", "#", "$", "%"])

def preprocess(text: str, only_kor: bool = True):
    """한국어 문장을 옵션에 맞게 전처리"""
    # 한국어 모음과 특수문자, 숫자 및 영어 제거
    if only_kor:
        text = re.sub(r"[^가-힣\s]+", "", text)
    else:
        text = re.sub(rf"[^가-힣ㄱ-ㅎ0-9{SPECIALS}a-zA-Z\s]+", "", text)

    # 연속 공백 제거
    text = re.sub(r"\s+", " ", text)

    # 좌우 불필요한 공백 제거
    return text.strip()


df_filtered['dialect_form'] = df_filtered['dialect_form'].apply(preprocess)
df_filtered['standard_form'] = df_filtered['standard_form'].apply(preprocess)

In [8]:
import sentencepiece as spm
from tokenizers import Tokenizer, normalizers
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from tokenizers.processors import TemplateProcessing
from datasets import Dataset
from transformers import PreTrainedTokenizerFast, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments


In [9]:

from transformers import PreTrainedTokenizerFast, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, BertTokenizer, AutoTokenizer
# Define custom tokenizer class
class CustomSPTokenizer(PreTrainedTokenizerFast):
    def __init__(self, tokenizer_path):
        super().__init__(tokenizer_file=tokenizer_path)
        self._tokenizer = Tokenizer.from_file(tokenizer_path)

    def _tokenize(self, text):
        return self._tokenizer.encode(text).tokens

    def _convert_token_to_id(self, token):
        return self._tokenizer.token_to_id(token)

    def _convert_id_to_token(self, index):
        return self._tokenizer.id_to_token(index)

    def convert_tokens_to_string(self, tokens):
        return self._tokenizer.decode(tokens)

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep + [self.sep_token_id]

    @property
    def cls_token_id(self):
        return self._tokenizer.token_to_id("<cls>")

    @property
    def sep_token_id(self):
        return self._tokenizer.token_to_id("<sep>")

    @property
    def pad_token_id(self):
        return self._tokenizer.token_to_id("<pad>")

    @property
    def unk_token_id(self):
        return self._tokenizer.token_to_id("<unk>")

# Initialize the custom tokenizer
tokenizer_path = "bpe_tokenizer.json"
custom_tokenizer = CustomSPTokenizer(tokenizer_path)
# kogpt2_tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', pad_token='<pad>')
# custom_tokenizer = CustomSPTokenizer(kogpt2_tokenizer)

# Add padding token to tokenizer
custom_tokenizer.add_special_tokens({'pad_token': '<pad>'})



0

In [10]:

# Update model with new token embeddings 
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")
model = AutoModelForSeq2SeqLM.from_pretrained("gogamza/kobart-base-v2")

# model.resize_token_embeddings(len(custom_tokenizer))

C:\Users\user\anaconda3\envs\koreanNLP\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [11]:
model.resize_token_embeddings(len(custom_tokenizer))

Embedding(16000, 768)

In [12]:
df_sampled = df_filtered

In [12]:
# # 3/10만 사용함. 
# df_sampled = df_filtered.sample(frac=0.3, random_state=42)  # Adjust the fraction as needed
# # df_sampled.info()
# df_sampled

# 길이 순 정렬
df_sampled = df_filtered
df_sampled['standard_form_length'] = df_sampled['standard_form'].apply(lambda x: len(x))
df_sampled = df_sampled.sort_values('standard_form_length')
df_sampled = df_sampled.drop('standard_form_length', axis=1)
# df_sampled.info()
df_sampled

,standard_form,dialect_form
820082,,암장
227859,,아닐케라
742561,,그 여자 대단하다이
819994,,지치못허게
243337,야,라
...,...,...
224452,아니 근데 나 진짜 웃긴 게 내가 그 다문화 그 아니 다문화가 아니라 아 다문화였나...,아니 근데 나 진짜 웃긴 게 내가 그 다문화 그 아니 다문화가 아니라 아 다문화였나...
94492,왜냐하면 진짜 이렇게 민원 넣기 쉽 쉽고 이러니깐 특히 특히 우리 한 여기 뭐냐 일...,왜냐하면 진짜 이렇게 민원 넣기 쉽 쉽고 이러니깐 특히 특히 우리 한 여기 뭐냐 일...
656625,빗대어서 아우 직설적으로 말하면게게 지 여자친구 싫다고 하는 거니깐 그건 안 되고 ...,빗대어그냉 아우 직설적으로 말하면게게 지 여자친구 싫댄 하는 거난 그건 안 되고 어...
94484,칠십팔 그래서 아 내가 딱 봐도 이게 컷이 그때는 문제가 원래 한자 시험이 진짜 개...,칠십팔 그래서 아 내가 딱 봐도 이게 컷이 그때는 문제가 원래 한자 시험이 진짜 개...


In [ ]:
import pandas as pd
import spacy

# spaCy 한국어 모델 로드
nlp = spacy.load('ko_core_news_sm')

# 데이터 로드 (CSV 파일에서 데이터를 불러온다고 가정)
df  = df_sampled #pd.read_csv('your_data.csv', encoding='utf-8')

def calculate_similarity(row):
    # 표준어와 방언 문장을 spaCy 문서 객체로 변환
    doc1 = nlp(row['standard_form'])
    doc2 = nlp(row['dialect_form'])
    
    # 두 문장 간의 유사도 계산
    similarity = doc1.similarity(doc2)
    
    return similarity

# 유사도 계산 및 새로운 열 추가
df['similarity'] = df.apply(calculate_similarity, axis=1)

# 유사도를 기준으로 내림차순 정렬
df_sorted = df.sort_values(by='similarity', ascending=False)



In [ ]:
# 결과 출력
print(df_sorted[['standard_form','dialect_form', 'similarity']])

# 결과를 CSV 파일로 저장
df_sorted.to_csv('jeju_dialect_similarity_results_sorted.csv', encoding='utf-8-sig', index=False)

In [14]:
df_sampled.to_csv('jeju.csv', index=False, encoding='cp949')

In [13]:
# df_sampled = df_filtered.sample(frac=0.3, random_state=42)  # Adjust the fraction as needed
# df_sampled['standard_form_length'] = df_sampled['standard_form'].apply(lambda x: len(x))
# df_sampled = df_sampled.sort_values('standard_form_length')
# df_sampled = df_sampled.drop('standard_form_length', axis=1)
# # df_sampled.info()
# df_sampled

In [14]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and evaluation sets
train_df, eval_df = train_test_split(df_sampled, test_size=0.2, random_state=42)   

# Convert to Hugging Face datasets format
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)
#########################################################################################################################
# Preprocess the data
def preprocess_function(examples):
    inputs = [ex for ex in examples['standard_form']]
    targets = [ex for ex in examples['dialect_form']]
    # inputs = [ex for ex in examples['dialect_form']]
    # targets = [ex for ex in examples['standard_form']]
    model_inputs = custom_tokenizer(inputs, max_length=100, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with custom_tokenizer.as_target_tokenizer():
        labels = custom_tokenizer(targets, max_length=100, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
#########################################################################################################################
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Filter out unwanted columns
train_dataset = train_dataset.remove_columns(['standard_form', 'dialect_form', '__index_level_0__', 'token_type_ids','similarity'])
eval_dataset = eval_dataset.remove_columns(['standard_form', 'dialect_form', '__index_level_0__', 'token_type_ids','similarity'])
print(train_dataset.column_names)  
print(eval_dataset.column_names)

Map:   0%|          | 0/351003 [00:00<?, ? examples/s]

Map:   0%|          | 0/87751 [00:00<?, ? examples/s]

['input_ids', 'attention_mask', 'labels']
['input_ids', 'attention_mask', 'labels']


In [15]:
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [16]:
# Load the BART model
# model = AutoModelForSeq2SeqLM.from_pretrained("gogamza/kobart-base-v2")
model.to(device)

# 로그 디렉토리 경로 설정
log_dir = r"C:\Users\user\Desktop\logs"
os.makedirs(log_dir, exist_ok=True)


training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",  # 'epoch' 대신 'steps'로 설정하여 주기적으로 평가
    eval_steps=500,  # 평가 주기를 500 스텝으로 설정 BART는 5000주기
    save_steps=1000,  # 체크포인트 저장 주기를 1000 스텝으로 설정
    learning_rate=2e-5,  # 일반적으로 2e-5와 같은 낮은 학습률이 안정적
    per_device_train_batch_size=32,  # GPU 메모리와 성능에 따라 조정  Bart = 16. KoBART = 32
    per_device_eval_batch_size=32,  # 평가 시에도 배치 크기를 맞춤  Bart = 16. KoBART = 32
    num_train_epochs=3,  # 기본 에포크 수
    weight_decay=0.01,  # 가중치 감쇠
    logging_dir=log_dir,  # 로그 디렉토리
    logging_steps=100,  # 로그 주기를 100 스텝으로 설정  Bart = 500. KoBART = 100
    save_total_limit=2,  # 저장할 체크포인트 수 제한
    load_best_model_at_end=True,  # 학습이 끝난 후 가장 좋은 모델을 로드
    gradient_accumulation_steps=2,  # 그래디언트 누적을 통해 배치 크기 증가 효과
)


# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=custom_tokenizer,
)


In [17]:
    learning_rate=2e-5,
    per_device_train_batch_size=32,   #Bart = 16. KoBART = 32
    per_device_eval_batch_size=32,    #Bart = 16. KoBART = 32
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir=log_dir, 
    logging_steps=100,                # 로그 주기를 100 스텝으로 설정  Bart = 500. KoBART = 100
    save_total_limit=2,
    load_best_model_at_end=True,
    gradient_accumulation_steps=2, 

In [18]:
trainer.train()

# 모델 저장
model.save_pretrained("./kobart_highsimilar_standard_to_dialect")
# model.save_pretrained("./translated_model_bart100_sort_dialect_to_standard")

C:\Users\user\anaconda3\envs\koreanNLP\lib\site-packages\transformers\models\bart\modeling_bart.py:587: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss,Validation Loss
500,0.200600,0.170716
1000,0.146900,0.137010
1500,0.126500,0.120547
2000,0.112300,0.110719
2500,0.105300,0.103277
3000,0.099000,0.100270
3500,0.096400,0.094459
4000,0.093100,0.089762
4500,0.087800,0.086961
5000,0.087300,0.086102


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

In [19]:
# 학습 루프가 끝난 후
# 모델 전체 저장 (구조와 가중치 모두 포함)
torch.save(model, "kobart_highsimilar_standard_to_dialect.pt")

In [16]:
# 학습 시작 또는 재개
trainer.train(resume_from_checkpoint=r'C:\Users\user\Desktop\2024-1\자연어처리\results\checkpoint-24000')

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
C:\Users\user\anaconda3\envs\koreanNLP\lib\site-packages\transformers\models\bart\modeling_bart.py:587: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss,Validation Loss
24500,0.052300,0.050489
25000,0.050500,0.050493
25500,0.049500,0.050143
26000,0.051600,0.050001
26500,0.052400,0.049661
27000,0.051800,0.049751
27500,0.051200,0.049291
28000,0.047300,0.049387
28500,0.047400,0.048877
29000,0.048500,0.048644


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

TrainOutput(global_step=41865, training_loss=0.020505461680850687, metrics={'train_runtime': 23483.7536, 'train_samples_per_second': 114.093, 'train_steps_per_second': 1.783, 'total_flos': 1.59539747447808e+17, 'train_loss': 0.020505461680850687, 'epoch': 3.0})

In [15]:
# 학습 시작 또는 재개
trainer.train(resume_from_checkpoint=r'C:\Users\user\Desktop\2024-1\자연어처리\results\checkpoint-80000')

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
C:\Users\user\anaconda3\envs\koreanNLP\lib\site-packages\transformers\models\bart\modeling_bart.py:587: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss,Validation Loss


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=83730, training_loss=0.003422350776988546, metrics={'train_runtime': 818.1575, 'train_samples_per_second': 3274.838, 'train_steps_per_second': 102.34, 'total_flos': 1.59539747447808e+17, 'train_loss': 0.003422350776988546, 'epoch': 3.0})

In [15]:
# 학습 시작 또는 재개
trainer.train(resume_from_checkpoint=r'C:\Users\user\Desktop\2024-1\자연어처리\results\checkpoint-80000')

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
C:\Users\user\anaconda3\envs\koreanNLP\lib\site-packages\transformers\models\bart\modeling_bart.py:587: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss,Validation Loss


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=83730, training_loss=0.003422350776988546, metrics={'train_runtime': 818.1575, 'train_samples_per_second': 3274.838, 'train_steps_per_second': 102.34, 'total_flos': 1.59539747447808e+17, 'train_loss': 0.003422350776988546, 'epoch': 3.0})

In [17]:

# 모델 저장
model.save_pretrained("./translated_model_kobart100_sort_dialect_to_standard")
# model.save_pretrained("./translated_model_bart100_sort_dialect_to_standard")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}


In [20]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# 모델과 토크나이저 로드 (모델이 저장된 경로로 수정)
model = AutoModelForSeq2SeqLM.from_pretrained("gogamza/kobart-base-v2")


def translate(text):
    # 입력 텍스트를 토크나이징
    inputs = custom_tokenizer(text, return_tensors="pt")
    # token_type_ids 키 제거
    inputs = {key: value for key, value in inputs.items() if key != 'token_type_ids'}
    print(len(inputs['input_ids'][0]))
    
    # 예측 생성
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=50,
        # max_length=len(inputs['input_ids'][0]),
        num_beams=4,
        # no_repeat_ngram_size=15,  # 반복 방지 설정
        early_stopping=True
    )
   
    # 생성된 토큰을 텍스트로 디코딩
    translated_text = custom_tokenizer.decode(outputs[0], skip_special_tokens=True)
    output = custom_tokenizer(translated_text, return_tensors="pt")
    print(output)
    return translated_text


# 예제 번역
# sample_sentence = "잡아서 온 거 보난 한 대여섯 마리 잡아서 온 것같아."
# sample_sentence = "그렇게 공부하면 시험 잘보겠어. 화이팅하자"
sample_sentence = "그렇게 귤이 좋으면 귤만 먹고 살아"
translated_sentence = translate(sample_sentence)
print(len(translated_sentence))
print(f"Standard: {sample_sentence}")
print(f"Dialect: {translated_sentence}")

C:\Users\user\anaconda3\envs\koreanNLP\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


10
{'input_ids': tensor([[    2,  9993, 14034, 14034, 14035, 14035, 14035, 14035, 14035, 14034,
         14035, 14035, 14034, 14034, 14034, 14034, 14035, 14035, 14034, 14034,
         14034, 14034, 14034, 14035, 14035, 14035, 14035, 14035, 14035,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}
139
Standard: 그렇게 귤이 좋으면 귤만 먹고 살아
Dialect: 3571 9417 9417 9418 9418 9418 9418 9418 9417 9418 9418 9417 9417 9417 9417 9418 9418 9417 9417 9417 9417 9417 9418 9418 9418 9418 9418 9418


In [120]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# 모델과 토크나이저 로드 (모델이 저장된 경로로 수정)
model = AutoModelForSeq2SeqLM.from_pretrained("./translated_model_kobart100_sort_standart_to_dialect")


def translate(text):
    # 입력 텍스트를 토크나이징
    inputs = custom_tokenizer(text, return_tensors="pt")
    # token_type_ids 키 제거
    inputs = {key: value for key, value in inputs.items() if key != 'token_type_ids'}
    print(len(inputs['input_ids'][0]))
    
    # 예측 생성
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=50,
        # max_length=len(inputs['input_ids'][0]),
        num_beams=2,
        early_stopping=True
    )
   
    # 생성된 토큰을 텍스트로 디코딩
    translated_text = custom_tokenizer.decode(outputs[0], skip_special_tokens=True)
    output = custom_tokenizer(translated_text, return_tensors="pt")
    print(output)
    return translated_text


# 예제 번역
# sample_sentence = "잡아서 온 거 보난 한 대여섯 마리 잡아서 온 것같아."
sample_sentence = "토라져 버리는거야"
# sample_sentence = "저는 이 사람이 좋습니다"
translated_sentence = translate(sample_sentence)
print(len(translated_sentence))
print(f"Standard: {sample_sentence}")
print(f"Dialect: {translated_sentence}")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


7
{'input_ids': tensor([[    2,  2477,   903,  2008,  1288, 14459,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
11
Standard: 토라져 버리는거야
Dialect: 토 라 져 부 는거라


In [19]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# 모델과 토크나이저 로드 (모델이 저장된 경로로 수정)
model = AutoModelForSeq2SeqLM.from_pretrained("./translated_model_kobart100_sort_dialect_to_standard")


def translate(text):
    # 입력 텍스트를 토크나이징
    inputs = custom_tokenizer(text, return_tensors="pt")
    # token_type_ids 키 제거
    inputs = {key: value for key, value in inputs.items() if key != 'token_type_ids'}
    print(len(inputs['input_ids'][0]))
    
    # 예측 생성
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=50,
        # max_length=len(inputs['input_ids'][0]),
        num_beams=2,
        no_repeat_ngram_size=15,  # 반복 방지 설정
        early_stopping=True
    )
   
    # 생성된 토큰을 텍스트로 디코딩
    translated_text = custom_tokenizer.decode(outputs[0], skip_special_tokens=True)
    output = custom_tokenizer(translated_text, return_tensors="pt")
    print(output)
    return translated_text


# 예제 번역
# sample_sentence = "잡아서 온 거 보난 한 대여섯 마리 잡아서 온 것같아."
sample_sentence = "겅 행 맨날 밥 하고 피곤 하겠다"
translated_sentence = translate(sample_sentence)
print(len(translated_sentence))
print(f"Standard: {sample_sentence}")
print(f"Dialect: {translated_sentence}")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


9
{'input_ids': tensor([[   2, 3261, 2929, 5649, 1191, 2906, 7602, 4262,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
21
Standard: 겅 행 맨날 밥 하고 피곤 하겠다
Dialect: 그렇게 해서 맨날 밥 하고 피곤 하겠다


In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# 모델과 토크나이저 로드 (모델이 저장된 경로로 수정)
model = AutoModelForSeq2SeqLM.from_pretrained("./translated_model_gy")


def translate(text):
    # 입력 텍스트를 토크나이징
    inputs = custom_tokenizer(text, return_tensors="pt")
    # token_type_ids 키 제거
    inputs = {key: value for key, value in inputs.items() if key != 'token_type_ids'}
    print(len(inputs['input_ids'][0]))
    
    # 예측 생성
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=50,
        # max_length=len(inputs['input_ids'][0]),
        num_beams=2,
        no_repeat_ngram_size=15,  # 반복 방지 설정
        early_stopping=True
    )
   
    # 생성된 토큰을 텍스트로 디코딩
    translated_text = custom_tokenizer.decode(outputs[0], skip_special_tokens=True)
    output = custom_tokenizer(translated_text, return_tensors="pt")
    print(output)
    return translated_text


# 예제 번역
sample_sentence = "잡아서 온 거 보난 한 대여섯 마리 잡아서 온 것같아."
# sample_sentence = "어제 그렇게 행복했어?"
translated_sentence = translate(sample_sentence)
print(len(translated_sentence))
print(f"Standard: {sample_sentence}")
print(f"Dialect: {translated_sentence}")

NameError: name 'custom_tokenizer' is not defined

In [41]:
# #model.save_pretrained("/content/drive/MyDrive/NLP/translated_model")
# # model = AutoModelForSeq2SeqLM.from_pretrained("./translated_model")


# def translate(text):
#     inputs = custom_tokenizer(text, return_tensors="pt")
#     inputs = {key: value for key, value in inputs.items() if key != 'token_type_ids'}  # Remove token_type_ids
#     print(inputs)
#     outputs = model.generate(**inputs, max_length=100, num_beams=4, early_stopping=True)
#     translated_text = custom_tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return translated_text

# # Example translation
# sample_sentence = "예전에 그렇게"
# translated_sentence = translate(sample_sentence)
# print(f"Standard: {sample_sentence}")
# print(f"Dialect: {translated_sentence}")

In [42]:
pip install sacrebleu

     ---------------------------------------- 0.0/58.0 kB ? eta -:--:--
     ------- -------------------------------- 10.2/58.0 kB ? eta -:--:--
     ------------- ------------------------ 20.5/58.0 kB 320.0 kB/s eta 0:00:01
     -------------------------- ----------- 41.0/58.0 kB 326.8 kB/s eta 0:00:01
     -------------------------------------- 58.0/58.0 kB 304.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/106.7 kB ? eta -:--:--
   -------------------------------------- - 102.4/106.7 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 106.7/106.7 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [43]:
from sacrebleu import corpus_bleu

def calculate_bleu(references, hypotheses):
    bleu_score = corpus_bleu(hypotheses, [references])
    return bleu_score.score

# 예시 사용법
reference_sentences = [
    "그렇게해서 맨날 밥 하고 피곤하겠다",
    "잡아서 온 거 보난 한 대여섯 마리 잡아서 온 것같아",
    "어제 그렇게 행복했어?"
]

translated_sentences = [
    "그렇게해서 맨날 밥 하고 피곤하겠다",
    "잡아서 온 거 보난 한 대 여섯 마리 잡아서 온 것 닮아.",
    "어제 겅 행복 핸?"
]

bleu_score = calculate_bleu(reference_sentences, translated_sentences)
print(f"BLEU 점수: {bleu_score:.2f}")

BLEU 점수: 44.26


### 여기야

In [55]:
# Sample a small portion of the dataset
df_sampled = df_filtered.sample(frac=0.1, random_state=42)  # Adjust the fraction as needed
df_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 111639 entries, 552182 to 964585
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   standard_form  111639 non-null  object
 1   dialect_form   111639 non-null  object
dtypes: object(2)
memory usage: 2.6+ MB


In [56]:
df_sampled.reset_index(drop=True, inplace=True)
df_sampled.head() 

,standard_form,dialect_form
0,그렇게해서 맨날 밥 하고 그걸로 밥하고 뭐 하고 다 -해-(해나시녜)/(했었어),경행 맨날 밥 하고 그걸로 밥하고 뭐 하고 다 -해-(해나시녜)/(했었어)
1,그 쎈 물들을 소는,그 쎈 물들을 쇠는
2,각자 메일로 보내자,각자 메일로 보내게
3,기계 아니야?,기계 아이가?
4,잡아서 온 거 보난 한 대여섯 마리 잡아서 온 것같아,잡앙 온 거 보난 혼 대여섯 마리 잡앙 온생이라


In [57]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and evaluation sets
train_df, eval_df = train_test_split(df_sampled, test_size=0.2, random_state=42)   ## 89311, 22328
# Convert to Hugging Face datasets format
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

In [59]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load tokenizer and model
kobart_tokenizer = PreTrainedTokenizerFast.from_pretrained('hyunwoongko/kobart')
kobart_model = BartForConditionalGeneration.from_pretrained('hyunwoongko/kobart')


# Tokenization function
def preprocess_function(examples):
    inputs = [ex for ex in examples['standard_form']]
    targets = [ex for ex in examples['dialect_form']]
    model_inputs = kobart_tokenizer(inputs, max_length=100, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with kobart_tokenizer.as_target_tokenizer():
        labels = kobart_tokenizer(targets, max_length=100, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Tokenize the dataset
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Map:   0%|          | 0/89311 [00:00<?, ? examples/s]

Map:   0%|          | 0/22328 [00:00<?, ? examples/s]

In [60]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
kobart_model.to(device)

cuda


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [62]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./kobart_results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,  # Adjust learning rate if needed
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()


  0%|          | 0/5582 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'loss': 0.3109, 'grad_norm': 1.2133976221084595, 'learning_rate': 4.552131852382659e-05, 'epoch': 0.09}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'loss': 0.2179, 'grad_norm': 0.9054189920425415, 'learning_rate': 4.104263704765317e-05, 'epoch': 0.18}


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'loss': 0.1419, 'grad_norm': 0.6197181940078735, 'learning_rate': 7.3450376209244e-07, 'epoch': 0.99}


  0%|          | 0/1396 [00:00<?, ?it/s]

{'eval_loss': 0.13399171829223633, 'eval_runtime': 125.7849, 'eval_samples_per_second': 177.509, 'eval_steps_per_second': 11.098, 'epoch': 1.0}
{'train_runtime': 2052.9032, 'train_samples_per_second': 43.505, 'train_steps_per_second': 2.719, 'train_loss': 0.177384459728923, 'epoch': 1.0}


TrainOutput(global_step=5582, training_loss=0.177384459728923, metrics={'train_runtime': 2052.9032, 'train_samples_per_second': 43.505, 'train_steps_per_second': 2.719, 'total_flos': 5317985627136000.0, 'train_loss': 0.177384459728923, 'epoch': 1.0})

In [63]:
kobart_model.save_pretrained("./kobart_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}


In [70]:
kobart_model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,